In [42]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

In [43]:
costs = pd.DataFrame({
    "A": [10, 12, 6],
    "B": [5, 7, 12],
    "C": [20, 12, 16],
    "D": [12, 19, 17],
}, index=[1, 2, 3])
costs

,A,B,C,D
1,10,5,20,12
2,12,7,12,19
3,6,12,16,17


In [73]:
supply = [11, 25, 14]
demand = [5, 15, 13, 17]
c = [10, 5, 20, 12, 12, 7, 12, 19, 6, 12, 16, 17]
num_supply = len(supply)
num_demand = len(demand)
A_eq = np.zeros((num_supply + num_demand, num_supply * num_demand))
for i in range(num_supply):
    A_eq[i, i * num_demand:(i + 1)* num_demand] = 1
for j in range(num_demand):
    A_eq[num_supply + j, j::num_demand] = 1
print(A_eq)
A_eq = A_eq.tolist()
b_eq = supply + demand
X_bounds = [(0, None)] * num_supply * num_demand
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=X_bounds, method='highs')
print(result.x)
print(np.array(result.x).reshape(num_supply, num_demand))

[[1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]
 [1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1.]]
[ 0.  3.  0.  8.  0. 12. 13.  0.  5.  0.  0.  9.]
[[ 0.  3.  0.  8.]
 [ 0. 12. 13.  0.]
 [ 5.  0.  0.  9.]]


In [66]:
c = [3,7,4,6,5,5,4,2,5,1,6,3,2,2,4]
num_stores = 5
num_warehouse = 3
A_eq = np.zeros((num_warehouse + num_stores, num_warehouse * num_stores))

for i in range(num_warehouse):
    A_eq[i, i*num_stores:(i+1)*num_stores] = 1
for j in range(num_stores):
    A_eq[num_warehouse +j, j::num_stores ] = 1
A_eq  = A_eq.tolist()
b_eq = [150, 110, 90, 50, 100, 70, 70, 60]

X_bounds = [(0, None)] * num_stores * num_warehouse
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=X_bounds, method='highs')
print(result.fun)
print(result.x)

1100.0
[50.  0. 70. 30.  0.  0. 50.  0.  0. 60.  0. 50.  0. 40.  0.]


In [3]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo

# Problem Statement Parameters
c = [3,7,4,6,5,5,4,2,5,1,6,3,2,2,4]
warehouse_supply = [150, 110, 90]
store_demand = [70,70, 60, 50, 100]

num_warehouse = len(warehouse_supply)
num_stores = len(store_demand)

# Pyomo model
model = pyo.ConcreteModel()
model.I = pyo.RangeSet(num_warehouse)
model.J = pyo.RangeSet(num_stores)

cost_matrix = {(i, j): c[(i - 1) * num_stores + (j -1)] for i in model.I for j in model.J }

# Parameters for supply and demand
model.supply = pyo.Param(model.I, initialize={i: warehouse_supply[i -1] for i in model.I})
model.demand = pyo.Param(model.J, initialize={j: store_demand[j -1] for j in model.J})

# Define variables
model.x = pyo.Var(model.I, model.J, domain=pyo.NonNegativeReals)

def objective_rule(model):
    return sum(cost_matrix[i, j] * model.x[i, j] for i in model.I for j in model.J)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

def supply_rule(model, i):
    return sum(model.x[i, j] for j in model.J) <= model.supply[i]
model.supply_constraint = pyo.Constraint(model.I, rule=supply_rule)

def demand_rule(model, j):
    return sum(model.x[i, j] for i in model.I) >= model.demand[j]
model.demand_constraint = pyo.Constraint(model.J, rule=demand_rule)

solver = pyo.SolverFactory('cbc')
result = solver.solve(model, tee=True)
print(f'Optimal cost: {pyo.value(model.objective)}')
for i in model.I:
    for j in model.J:
        print(f'x[{i}{j}] = {pyo.value(model.x[i, j])}')

Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Sep  3 2024 

command line - /opt/anaconda3/envs/learning/bin/cbc -printingOptions all -import /var/folders/j0/tr2332417vjbmj74tfc0dcd40000gn/T/tmpvire78nd.pyomo.lp -stat=1 -solve -solu /var/folders/j0/tr2332417vjbmj74tfc0dcd40000gn/T/tmpvire78nd.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 8 (0) rows, 15 (0) columns and 30 (0) elements
Statistics for presolved model


Problem has 8 rows, 15 columns (15 with objective) and 30 elements
Column breakdown:
15 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
5 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
3 of type L other, 0 of type Range 0.0->1.0, 0 of type Range other, 
0 of type Fre

In [1]:
warehouse = [150, 110, 90, 50, 100]
dict(zip(range(len(warehouse)), warehouse))

{0: 150, 1: 110, 2: 90, 3: 50, 4: 100}

In [57]:
import numpy as np
from scipy.optimize import linprog

# Step 1: Define the costs, supply, and demand (adjusted)
supply = [14, 25, 11]  # Adjusted total supply to 50
demand = [5, 15, 13, 17]  # Total demand remains 50
c = [10, 5, 20, 12, 12, 7, 12, 19, 6, 12, 16, 17]  # Flattened cost matrix

num_supply = len(supply)  # Number of suppliers
num_demand = len(demand)  # Number of customers

# Step 2: Create the equality constraint matrix (A_eq)
A_eq = np.zeros((num_supply + num_demand, num_supply * num_demand))

# Supply constraints
for i in range(num_supply):
    for j in range(num_demand):
        A_eq[i, i * num_demand + j] = 1

# Demand constraints
for j in range(num_demand):
    for i in range(num_supply):
        A_eq[num_supply + j, i * num_demand + j] = 1

# Step 3: Create the right-hand side vector (b_eq)
b_eq = supply + demand

# Step 4: Set bounds for decision variables (non-negative)
X_bounds = [(0, None)] * (num_supply * num_demand)

# Step 5: Solve the transportation problem
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=X_bounds, method="highs")

# Step 6: Display results
if result.success:
    print("Optimal solution found!")
    print("Total Cost:", result.fun)
    print("Transportation Plan:")
    print(np.array(result.x).reshape(num_supply, num_demand))
else:
    print("No feasible solution found.")

Optimal solution found!
Total Cost: 519.0
Transportation Plan:
[[ 0.  3.  0. 11.]
 [ 0. 12. 13.  0.]
 [ 5.  0.  0.  6.]]
